In [1]:
import veloxchem as vlx

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 8.


In [2]:
class MockTask:
    def __init__(self, mol, basis, comm, ostream):

        self.molecule = mol
        self.ao_basis = basis
        self.mpi_comm = comm
        self.ostream = ostream

In [3]:
import adcc
from mpi4py import MPI
import sys
import numpy
numpy.set_printoptions(precision=7, suppress=True)

In [4]:
#Input settings
molecule_string = """
    O 0 0 0
    H 0 0 1.795239827225189
    H 1.693194615993441 0 -0.599043184453037"""

# molecule_string = """
#    Li 0 0 0
#    H 0 0 1.795239827225189
#    """

basis_set_label = 'STO-3G'
scf_settings = {'conv_thresh': 1.0e-8}
method_settings = {} #{'xcfun': 'b3lyp', 'grid_level': 4}
rsp_settings = {'conv_thresh': 1.0e-6, 'nstates': 3}

In [5]:
# Communicator and output stream
comm = MPI.COMM_WORLD
ostream = vlx.OutputStream(sys.stdout)

In [6]:
# Molecule and basis set
molecule = vlx.Molecule.read_str(molecule_string, units='au')
basis = vlx.MolecularBasis.read(molecule, basis_set_label)

ostream.print_block(molecule.get_string())
ostream.print_block(basis.get_string('Atomic Basis', molecule))
ostream.flush()

                                              Molecular Geometry (Angstroms)                                              
                                                                                                                          
                          Atom         Coordinate X          Coordinate Y          Coordinate Z                           
                                                                                                                          
                           O           0.000000000000        0.000000000000        0.000000000000                         
                           H           0.000000000000        0.000000000000        0.950000004704                         
                           H           0.896000004436        0.000000000000       -0.317000001569                         
                                                                                                                          
                

In [7]:
# SCF
task = MockTask(molecule, basis, comm, ostream)
scfdrv = vlx.ScfRestrictedDriver(comm, ostream)
scfdrv.update_settings(scf_settings, method_settings)
scfdrv.compute(molecule, basis)
scfdrv.task = task

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

In [8]:
# SCF first-order properties
scf_prop = vlx.ScfFirstOrderProperties(comm, ostream)
scf_prop.compute(molecule, basis, scfdrv.scf_tensors)
scf_prop.print_properties(molecule)

                                                                                                                          
                                                Ground-State Dipole Moment                                                
                                               ----------------------------                                               
                                                                                                                          
                                   X   :         0.546315 a.u.         1.388593 Debye                                     
                                   Y   :         0.000000 a.u.         0.000000 Debye                                     
                                   Z   :         0.386302 a.u.         0.981881 Debye                                     
                                 Total :         0.669095 a.u.         1.700671 Debye                                     
                

In [9]:
# Linear Response
#lr_eig_drv = vlx.LinearResponseEigenSolver(comm, ostream)
#lr_eig_drv.update_settings(rsp_settings, method_settings)
#lr_results = lr_eig_drv.compute(molecule, basis, scfdrv.scf_tensors)
#lr_eig_drv.ostream.flush()

In [10]:
# TDHF/TDA, i.e. CIS
tda_drv = vlx.TDAExciDriver(comm, ostream)
tda_drv.update_settings(rsp_settings, method_settings)
tda_results = tda_drv.compute(molecule, basis, scfdrv.scf_tensors)
tda_drv.ostream.flush()

                                                                                                                          
                                                     TDA Driver Setup                                                     
                                                                                                                          
                               Number of States                : 3                                                        
                               Max. Number of Iterations       : 150                                                      
                               Convergence Threshold           : 1.0e-06                                                  
                               ERI Screening Scheme            : Cauchy Schwarz + Density                                 
                               ERI Screening Threshold         : 1.0e-15                                                  
                

In [11]:
# Get MO coefficients, eigenvalues and eigenvectors

nocc = molecule.number_of_alpha_electrons()
mo = scfdrv.scf_tensors['C'] # MO coefficients
mo_occ = mo[:, :nocc]        # occupied
mo_vir = mo[:, nocc:]        # virtual

nocc = mo_occ.shape[1]
nvir = mo_vir.shape[1]

#eig_vals = lr_results["eigenvalues"]
#eig_vecs = lr_results["eigenvectors"]

#eig_vec = eig_vecs[:, 0].copy() # first eigenvector
#print(eig_vec)
print("mo_occ:")
print(mo_occ)
print()
print("mo_vir:")
print(mo_vir)

mo_occ:
[[ 0.9941004 -0.2328975 -0.0000651  0.100218  -0.       ]
 [ 0.0267754  0.8318658  0.0003549 -0.5233255  0.       ]
 [-0.0060845  0.1602966 -0.4415423  0.2690376 -0.       ]
 [-0.0060786  0.1601298  0.4415633  0.2692782 -0.       ]
 [ 0.        -0.        -0.         0.         1.       ]
 [ 0.0024519  0.0731199 -0.4902286  0.4582281 -0.       ]
 [ 0.0034627  0.103217   0.3463593  0.6483629 -0.       ]]

mo_vir:
[[-0.1356068 -0.0002233]
 [ 0.9082267  0.0016893]
 [-0.8061612 -0.8437322]
 [-0.8082192  0.8410846]
 [-0.        -0.       ]
 [ 0.4112391  0.8244282]
 [ 0.5840569 -0.5814892]]


In [12]:
print(scfdrv.scf_tensors.keys())
# Transform the Fock matrix from AO to MO basis
fock = scfdrv.scf_tensors['F'][0]
ovlp = scfdrv.scf_tensors['S']
print(numpy.matmul(mo.T, numpy.matmul(ovlp, mo)))
print(numpy.matmul(mo.T, numpy.matmul(fock, mo)))

dict_keys(['C', 'E', 'S', 'D', 'F'])
[[ 1.  0.  0.  0.  0.  0.  0.]
 [ 0.  1. -0.  0. -0. -0. -0.]
 [ 0. -0.  1. -0. -0. -0.  0.]
 [ 0. -0. -0.  1.  0. -0.  0.]
 [ 0. -0. -0.  0.  1.  0.  0.]
 [ 0. -0. -0. -0.  0.  1. -0.]
 [ 0. -0.  0.  0.  0. -0.  1.]]
[[-20.2333974   0.         -0.          0.         -0.          0.
    0.       ]
 [  0.         -1.2657146  -0.          0.         -0.         -0.
    0.       ]
 [ -0.         -0.         -0.6292671  -0.          0.          0.
    0.       ]
 [  0.          0.         -0.         -0.441668   -0.         -0.
    0.       ]
 [ -0.         -0.          0.         -0.         -0.3876451   0.
    0.       ]
 [  0.         -0.          0.         -0.          0.          0.6028394
    0.       ]
 [  0.          0.          0.          0.          0.          0.
    0.7659183]]


In [13]:
# Convert the first eigenvector to matrix form
#eig_vec_as_mat = vlx.LinearResponseEigenSolver.lrvec2mat(eig_vec, nocc, nocc + nvir)

In [14]:
tda_eig_vals = tda_results["eigenvalues"]
tda_eig_vecs = tda_results["eigenvectors"]
tda_size = tda_eig_vecs[:,0].shape
tda_eig_vec=tda_eig_vecs[:,0].copy()
tda_eig_vec_as_mat = tda_eig_vec.reshape(nocc, nvir) #/ numpy.sqrt(2.0)

In [15]:
print(tda_eig_vec)

[-0.        -0.        -0.         0.        -0.         0.
  0.         0.         0.9999995 -0.0009539]


In [16]:
#tda_eig_vec_as_mat = vlx.LinearResponseEigenSolver.lrvec2mat(tda_eig_vec, nocc, nocc + nvir)

In [17]:
print(tda_eig_vec_as_mat)

[[-0.        -0.       ]
 [-0.         0.       ]
 [-0.         0.       ]
 [ 0.         0.       ]
 [ 0.9999995 -0.0009539]]


In [18]:
adc1 = adcc.run_adc(scfdrv, method='adc1', n_singlets=3, conv_tol=1e-4)

Starting adc1 singlet Jacobi-Davidson ...
Niter n_ss  max_residual  time  Ritz values
  1     6     0.0028135   13ms  [0.4834361 0.5742004 0.6052005]
  2    10    9.4737e-30   18ms  [0.4834361 0.5742004 0.602137 ]
=== Converged ===
    Number of matrix applies:    10
    Total solver time:           34.000ms


In [19]:
adc1_eig_vec = adc1.excitation_vector[0]["s"].to_ndarray()
adc1_eig_vals = adc1.excitation_energy
adc1.state_dipole_moment

/home/manu/.local/lib/python3.8/site-packages/adcc/AmplitudeVector.py:78: UserWarning: Using the list interface of AmplitudeVector is deprecated and will be removed in version 0.16.0. Use block labels like 'ph', 'pphh' instead.
  warnings.warn("Using the list interface of AmplitudeVector is "


array([[-0.2107819, -0.       , -0.1423891],
       [ 0.0418872,  0.       ,  0.0233941],
       [-0.3315491,  0.       , -0.2288043]])

In [20]:
print(adc1_eig_vec)
print(adc1_eig_vals)
print(adc1.describe())

[[-0.        -0.         0.         0.       ]
 [ 0.        -0.         0.         0.       ]
 [ 0.        -0.         0.         0.       ]
 [-0.        -0.         0.         0.       ]
 [-0.7071065  0.0006745  0.         0.       ]
 [ 0.         0.        -0.        -0.       ]
 [ 0.         0.         0.        -0.       ]
 [ 0.         0.         0.        -0.       ]
 [ 0.         0.        -0.        -0.       ]
 [ 0.         0.        -0.7071065  0.0006745]]
[0.4834361 0.5742004 0.602137 ]
+----------------------------------------------------+
| adc1                          singlet ,  converged |
+----------------------------------------------------+
|  #        excitation energy     osc str    |v1|^2  |
|          (au)           (eV)                       |
|  0     0.4834361      13.15497   0.0038         1  |
|  1     0.5742004      15.62479   0.0000         1  |
|  2      0.602137      16.38498   0.0546         1  |
+----------------------------------------------------+



In [21]:
print(tda_eig_vecs[:,0])
print(tda_eig_vals)

[-0.        -0.        -0.         0.        -0.         0.
  0.         0.         0.9999995 -0.0009539]
[0.4834361 0.5742004 0.602137 ]


In [22]:
numpy.linalg.norm(adc1_eig_vec)

0.9999999999999999

In [23]:
0.9999995/numpy.sqrt(2.0)

0.7071064276331569

In [24]:
0.0009539/numpy.sqrt(2.0)

0.0006745091585738476

In [25]:
print(adc1_eig_vec.shape)
print(adc1_eig_vec)
print()
print(tda_eig_vec_as_mat.shape)
print(tda_eig_vec_as_mat/numpy.sqrt(2))

(10, 4)
[[-0.        -0.         0.         0.       ]
 [ 0.        -0.         0.         0.       ]
 [ 0.        -0.         0.         0.       ]
 [-0.        -0.         0.         0.       ]
 [-0.7071065  0.0006745  0.         0.       ]
 [ 0.         0.        -0.        -0.       ]
 [ 0.         0.         0.        -0.       ]
 [ 0.         0.         0.        -0.       ]
 [ 0.         0.        -0.        -0.       ]
 [ 0.         0.        -0.7071065  0.0006745]]

(5, 2)
[[-0.        -0.       ]
 [-0.         0.       ]
 [-0.         0.       ]
 [ 0.         0.       ]
 [ 0.7071065 -0.0006745]]


In [26]:
vlx_dm_oo = -numpy.einsum('ia,ja->ij', tda_eig_vec_as_mat, tda_eig_vec_as_mat)
vlx_dm_vv = numpy.einsum('ia,ib->ab', tda_eig_vec_as_mat, tda_eig_vec_as_mat)
vlx_DM_ovov = -numpy.einsum('ib,ja->iajb', tda_eig_vec_as_mat, tda_eig_vec_as_mat)

print(vlx_dm_oo)

[[-0. -0. -0.  0.  0.]
 [-0. -0. -0.  0.  0.]
 [-0. -0. -0.  0.  0.]
 [ 0.  0.  0. -0. -0.]
 [ 0.  0.  0. -0. -1.]]


In [27]:
adcc_dm_oo = -numpy.einsum('ia,ja->ij', adc1_eig_vec, adc1_eig_vec)
adcc_dm_vv = numpy.einsum('ia,ib->ab', adc1_eig_vec, adc1_eig_vec)
adcc_DM_ovov = -numpy.einsum('ib,ja->iajb', adc1_eig_vec, adc1_eig_vec)

#print(adcc_dm_oo)

foo = adc1.reference_state.fock("o1o1").to_ndarray()
fvv = adc1.reference_state.fock("v1v1").to_ndarray()
ovov = adc1.reference_state.eri("o1v1o1v1").to_ndarray()

adcc_excitation_energy = (numpy.einsum('ij,ij->',adcc_dm_oo,foo)
                     + numpy.einsum('ab,ab->', adcc_dm_vv, fvv)
                     + numpy.einsum('iajb,iajb->', adcc_DM_ovov, ovov)
                    )

In [28]:
print(adcc_excitation_energy)

0.4834360910410622


$E = \frac{1}{4} \sum_{pqrs} \Gamma_{pqrs} \langle pq || rs \rangle$

$\langle ia || jb \rangle = - \langle ai || jb \rangle = - \langle ia || bj \rangle = \langle ai || bj \rangle$

analogous symmetry for $\boldsymbol{\Gamma}$

In [29]:
vlx_excitation_energy = (1.0*numpy.einsum('ij,ij->', vlx_dm_oo,foo[:nocc,:nocc])
                        +1.0*numpy.einsum('ab,ab->', vlx_dm_vv, fvv[:nvir,:nvir])
                        +1.0*(
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, :nocc, :nvir])##aaaa
                        #+0.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, :nocc, nvir:])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, nocc:, :nvir])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, nocc:, nvir:])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, :nocc, :nvir])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, :nocc, nvir:])# zero block
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, nocc:, :nvir])##bbbb
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, nocc:, nvir:])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, :nvir, :nocc, :nvir])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, :nvir, :nocc, nvir:])##abba
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, :nvir, nocc:, :nvir])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, nvir:, :nocc, :nvir])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, nvir:, :nocc, nvir:])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, nvir:, nocc:, :nvir])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, nvir:, nocc:, nvir:])##baab
                        ))
print("vlx energy =", vlx_excitation_energy)

vlx energy = 0.4834360910410628


In [30]:
print(vlx_excitation_energy)

0.4834360910410628


In [31]:
adcc_excitation_energy = (1.0*numpy.einsum('ij,ij->', vlx_dm_oo,foo[:nocc,:nocc])
                        +1.0*numpy.einsum('ab,ab->', vlx_dm_vv, fvv[:nvir,:nvir])
                        +2.0*numpy.einsum('iajb,iajb->', adcc_DM_ovov[:nocc, :nvir, :nocc, :nvir], ovov[:nocc, :nvir, :nocc, :nvir])
                        +2.0*numpy.einsum('iajb,iajb->', adcc_DM_ovov[:nocc, :nvir, :nocc, :nvir], ovov[:nocc, nvir:, nocc:, :nvir])
    )
print(adcc_excitation_energy)

0.4834360910410391


In [32]:
print(ovov.shape)
#print(ovov)

(10, 4, 10, 4)


In [33]:
#print(vlx_DM_ovov)
#print()
print(vlx_DM_ovov.shape)
#print(adcc_DM_ovov)

(5, 2, 5, 2)


In [34]:
# Making the Vlx 2PDM twice the size with the other spin blocks
# and assign the corresponding values to it by comparing to the adcc matrix
# afterwards divide by two to yield the same numbers
##vlx_DM_ovov_full = numpy.zeros((2*nocc, 2*nvir, 2*nocc, 2*nvir))
#vlx_DM_ovov_full[:nocc, :nvir, :nocc, :nvir] = vlx_DM_ovov
##vlx_DM_ovov_full[0, 0, 0, 0] = vlx_DM_ovov_full[0, 3, 1, 0] = vlx_DM_ovov_full[1,0,0,3] = vlx_DM_ovov_full[1,3,1,3] = vlx_DM_ovov[0,0,0,0]
##vlx_DM_ovov_full[0, 0, 0, 2] = vlx_DM_ovov_full[0, 2, 0, 0] = vlx_DM_ovov_full[0,3,1,2] = vlx_DM_ovov_full[0,5,1,0] = vlx_DM_ovov[0,2,0,0]
##vlx_DM_ovov_full[1, 0, 0, 5] = vlx_DM_ovov_full[1, 2, 0, 3] = vlx_DM_ovov_full[1,3,1,5] = vlx_DM_ovov_full[1,5,1,3] = vlx_DM_ovov[0,2,0,0]
##vlx_DM_ovov_full[0, 2, 0, 2] = vlx_DM_ovov_full[0, 5, 1, 2] = vlx_DM_ovov_full[1,2,0,5] = vlx_DM_ovov_full[1,5,1,5] = vlx_DM_ovov[0,2,0,2]

### matrix elements were twice as large
##vlx_DM_ovov_full *= 0.5
# norm of the difference between the two density matrices does not seem to vanish entirely...
##print("norm of the difference of the 2PDMs = ", numpy.linalg.norm(adcc_DM_ovov - vlx_DM_ovov_full))
#print(vlx_DM_ovov_full)
##vlx_excitation_energy = (1.0*numpy.einsum('ij,ij->', vlx_dm_oo,foo[:nocc,:nocc])
##                        #(1.0*numpy.einsum('ij,ij->', adcc_dm_oo,foo)
##                        #+1.0*numpy.einsum('ij,ij->', adcc_dm_vv,fvv)
##                        +1.0*numpy.einsum('ab,ab->', vlx_dm_vv, fvv[:nvir,:nvir])
##                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov_full, ovov)
##                        )
##print("vlx exc. energy =", vlx_excitation_energy)
#print("adcc - vlx =", adcc_excitation_energy - vlx_excitation_energy)

In [35]:
print(adcc_DM_ovov.shape)
print()
#print(adcc_DM_ovov)
print()
#print(adcc_DM_ovov - vlx_DM_ovov_full) # here everything appears to be zero

(10, 4, 10, 4)




In [36]:
# Transform the one-particle density matrices from MO to AO basis
vlx_dm_oo_in_ao = numpy.matmul(mo_occ, numpy.matmul(vlx_dm_oo, mo_occ.T))
adcc_dm_oo_in_ao = 2.0*numpy.matmul(mo_occ, numpy.matmul(adcc_dm_oo[:nocc,:nocc], mo_occ.T))

vlx_dm_vv_in_ao = numpy.matmul(mo_vir, numpy.matmul(vlx_dm_vv, mo_vir.T))
adcc_dm_vv_in_ao = 2.0*numpy.matmul(mo_vir, numpy.matmul(adcc_dm_vv[:nvir,:nvir], mo_vir.T))

vlx_dm_ao = vlx_dm_oo_in_ao + vlx_dm_vv_in_ao
adcc_dm_ao = adcc_dm_oo_in_ao + adcc_dm_vv_in_ao

print("VLX")
print(vlx_dm_ao)
print("\nADCC")
print(adcc_dm_ao)

contract_fock_mo = (1.0*numpy.einsum('ij,ij->', vlx_dm_oo, foo[:nocc,:nocc])
                    +1.0*numpy.einsum('ab,ab->', vlx_dm_vv, fvv[:nvir,:nvir]))

contract_fock_ao = numpy.einsum('mn,mn->', vlx_dm_ao, fock)

print("contract_fock_ao =", contract_fock_ao)
print("contract_fock_mo =", contract_fock_mo)

VLX
[[ 0.0183891 -0.1231612  0.1092115  0.1097085  0.        -0.05566
  -0.0792771]
 [-0.1231612  0.8248721 -0.7314442 -0.734773  -0.         0.3727831
   0.5309584]
 [ 0.1092115 -0.7314442  0.6485983  0.6515501  0.        -0.3305604
  -0.4708202]
 [ 0.1097085 -0.734773   0.6515501  0.6545153 -0.        -0.3320648
  -0.4729629]
 [ 0.        -0.         0.        -0.        -1.        -0.
   0.       ]
 [-0.05566    0.3727831 -0.3305604 -0.3320648 -0.         0.1684713
   0.2399552]
 [-0.0792771  0.5309584 -0.4708202 -0.4729629  0.         0.2399552
   0.3417704]]

ADCC
[[ 0.0183891 -0.1231612  0.1092115  0.1097085 -0.        -0.05566
  -0.0792771]
 [-0.1231612  0.8248721 -0.7314442 -0.734773   0.         0.3727831
   0.5309584]
 [ 0.1092115 -0.7314442  0.6485983  0.6515501  0.        -0.3305604
  -0.4708202]
 [ 0.1097085 -0.734773   0.6515501  0.6545153 -0.        -0.3320648
  -0.4729629]
 [-0.         0.         0.        -0.        -1.        -0.
  -0.       ]
 [-0.05566    0.3727831

In [37]:
# Calculate the total density (including HF part)
# and calculated the unrelaxed excited-state dipole moment

# dipole integrals
dipole_drv = vlx.ElectricDipoleIntegralsDriver(comm)
dipole_mats = dipole_drv.compute(molecule, basis)
dipole_ints = (dipole_mats.x_to_numpy(),
              dipole_mats.y_to_numpy(),
              dipole_mats.z_to_numpy())

# electronic contribution
total_density = 2*scfdrv.scf_tensors['D'][0] + vlx_dm_ao
adcc_total_density = 2*scfdrv.scf_tensors['D'][0] + adcc_dm_ao
electronic_dipole = -1.0 * numpy.array(
    [numpy.sum(dipole_ints[d] * total_density) for d in range(3)])
adcc_electronic_dipole = -1.0 * numpy.array(
    [numpy.sum(dipole_ints[d] * adcc_total_density) for d in range(3)])

# nuclear contribution
coords = molecule.get_coordinates()
nuclear_charges = molecule.elem_ids_to_numpy()
nuclear_dipole = numpy.sum(coords.T * nuclear_charges, axis=1)

dipole_moment = (nuclear_dipole + electronic_dipole)
adcc_dipole_moment = (nuclear_dipole + adcc_electronic_dipole)

print("dipole moment [au] =", dipole_moment)
print()
print("adcc dm dipole moment [au] =", adcc_dipole_moment)
print()
print("ADC(1) dipole moment for water in Q-Chem:")
print("Dip. moment [a.u.]:  [0.221200,  -0.00000,   0.151505]")
print()
print("ADC(1) dipole moment from adcc:")
print(adc1.state_dipole_moment)

dipole moment [au] = [-0.2107819  0.        -0.1423891]

adcc dm dipole moment [au] = [-0.2107819 -0.        -0.1423891]

ADC(1) dipole moment for water in Q-Chem:
Dip. moment [a.u.]:  [0.221200,  -0.00000,   0.151505]

ADC(1) dipole moment from adcc:
[[-0.2107819 -0.        -0.1423891]
 [ 0.0418872  0.         0.0233941]
 [-0.3315491  0.        -0.2288043]]


In [38]:
# Transform the excitation vectors from MO to AO basis
tda_eig_vec_ao = numpy.matmul(mo_occ, numpy.matmul(tda_eig_vec_as_mat, mo_vir.T))
print(tda_eig_vec_ao)

[[ 0.        -0.         0.         0.        -0.        -0.
  -0.       ]
 [-0.         0.        -0.        -0.         0.         0.
   0.       ]
 [ 0.        -0.         0.         0.         0.        -0.
  -0.       ]
 [-0.         0.        -0.        -0.        -0.         0.
   0.       ]
 [-0.1356065  0.9082247 -0.805356  -0.8090212 -0.         0.4104525
   0.5846113]
 [-0.         0.        -0.        -0.        -0.         0.
   0.       ]
 [ 0.        -0.         0.         0.        -0.        -0.
  -0.       ]]


In [39]:
#dir(scfdrv)

In [40]:
# Get the ERI tensor in AO basis (chemists' notation)

nao = mo.shape[0]

pqrs = numpy.zeros((nao, nao, nao, nao))
eri_drv = vlx.ElectronRepulsionIntegralsDriver(comm)
eri_drv.compute_in_mem(molecule, basis, pqrs)
#print("(pq|rs) =")
#print(pqrs)


\begin{align}
\begin{split}
    - \sum_{\sigma \sigma'} \sum_{ijab} x_{i_{\sigma} b_{\sigma}} x_{j_{\sigma'} a_{\sigma'}}
        \langle i_{\sigma} a_{\sigma'} || j_{\sigma'} b_{\sigma} \rangle
    &= - \sum_{ijab} x_{ib} x_{ja} [ 2 (ij|ab) - 4 (ib|aj) ] \\
    &= + \sum_{ijab} x_{ib} x_{ja} [ 4 (ib|aj) - 2 (ij|ab)  ] \\
    &= \sum_{ijab} x_{ib} x_{ja} \sum_{\mu \nu \theta \varphi}
        C_{\mu i} C_{\nu b} C_{\theta a} C_{\varphi j}  [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ] \\
    &= \sum_{\mu \nu \theta \varphi} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]
        \sum_{ib} C_{\mu i} x_{ib} C_{\nu b} \sum_{ja} C_{\varphi j} x_{ja} C_{\theta a} \\
    &= \sum_{\mu \nu \theta \varphi} x_{\mu \nu} x_{\varphi \theta} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]       \\
    &= \sum_{\mu \nu} x_{\mu \nu} \sum_{\theta \varphi} x_{\varphi \theta} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]
\end{split}
\end{align}

In [41]:
# Contraction of electron repulsion integrals in AO basis
# First reference value in MO
eri_contract_mo_adcc = numpy.einsum('iajb,iajb->', adcc_DM_ovov, ovov)
print("Contraction in MO adcc:", eri_contract_mo_adcc)

eri_contract_mo_vlx = (+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, :nocc, :nvir])##aaaa
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, nocc:, :nvir]))##bbbb
print("Contraction in MO vlx: ", eri_contract_mo_vlx)

# Now contract everything in AO
eri_contract_ao = (0.5*numpy.einsum('mn,mn->', tda_eig_vec_ao, numpy.einsum('pt,mntp->mn', tda_eig_vec_ao, 4*pqrs)
                                   - numpy.einsum('pt,mptn->mn', tda_eig_vec_ao, 2*pqrs))
                    )
print("Contraction in AO:     ", eri_contract_ao)

Contraction in MO adcc: -0.5070485183213385
Contraction in MO vlx:  -0.5070485183213148
Contraction in AO:      -0.5070485183213149


In [42]:
mo.shape
print(nocc + nvir)

7


## Orbital response, right hand side

### 1. Contributions from the 1PDMs
\begin{equation}
F^\gamma_{ia}=\sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}\sum_{\mu,\nu,\sigma}\gamma^\sigma_{\mu\nu}(\mu\nu|\theta \varphi)-\sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}\sum_{\mu,\nu}\gamma^{\alpha}_{\mu\nu}(\mu\varphi|\theta\nu)\\
=\sum_{\theta,\varphi,\sigma}C^\alpha_{\varphi i}C^\alpha_{\theta a}F^{1,\sigma}_{\varphi\theta}-\sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}F^{2,\alpha}_{\varphi\theta}\\
=\sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}F_{\varphi\theta}
\end{equation}

In [43]:
F1_vlx = numpy.einsum('mn,mntp->pt', vlx_dm_ao, pqrs)
F2_vlx = numpy.einsum('mn,mptn->pt', vlx_dm_ao, pqrs)
F_1pdm_vlx = F1_vlx - 0.5*F2_vlx
vlx_1pdm_rhs = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, F_1pdm_vlx))
print(vlx_1pdm_rhs)

[[ 0.018333  -0.0000249]
 [-0.0827309 -0.0000336]
 [-0.0004581  0.1069933]
 [-0.0907945  0.0001547]
 [ 0.         0.       ]]


In [44]:
ooov = adc1.reference_state.eri("o1o1o1v1").to_ndarray()
ovvv = adc1.reference_state.eri("o1v1v1v1").to_ndarray()
adcc_1pdm_rhs = ( -numpy.einsum('jk,ikja->ia', adcc_dm_oo, ooov, optimize=True) 
                 -numpy.einsum('bc,icba->ia', adcc_dm_vv, ovvv, optimize=True)
                )
print(adcc_1pdm_rhs)

[[ 0.018333  -0.0000249 -0.        -0.       ]
 [-0.0827309 -0.0000336 -0.        -0.       ]
 [-0.0004581  0.1069933 -0.        -0.       ]
 [-0.0907945  0.0001547 -0.        -0.       ]
 [ 0.         0.        -0.        -0.       ]
 [-0.        -0.         0.018333  -0.0000249]
 [-0.        -0.        -0.0827309 -0.0000336]
 [-0.        -0.        -0.0004581  0.1069933]
 [-0.        -0.        -0.0907945  0.0001547]
 [-0.        -0.         0.         0.       ]]


### 2. Contributions from the 2PDMs

\begin{equation}
F^\Gamma_{ia}=\sum_{\mu,\rho}C^\alpha_{\mu a}C^\alpha_{\rho i}\left[\sum_{\tau,\varphi}S_{\rho\tau}x^\alpha_{\tau\varphi}\sum_{\nu,\theta}x^\alpha_{\theta\nu}(\mu\nu|\theta\varphi)-\sum_{\tau,\nu}S_{\rho\tau}x^\alpha_{\tau\nu}\sum_{\theta,\varphi,\sigma} x^\sigma_{\theta\varphi}(\mu\nu|\theta\varphi)\right]-\sum_{\mu,\rho}C^\alpha_{\rho i}C^\alpha_{\mu a}\left[\sum_{\tau,\varphi}S_{\mu\tau}x^\alpha_{\tau\varphi}\sum_{\nu,\theta}x^\alpha_{\theta\nu}(\rho\nu|\theta\varphi)-\sum_{\tau,\nu} S_{\mu\tau}x^\alpha_{\tau\nu}\sum_{\theta,\varphi,\sigma}x^\sigma_{\theta\varphi}(\rho\nu|\theta\varphi) \right]
\end{equation}

Corrected Equation:

\begin{equation}
F^\Gamma_{ia}=\sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a}\left[\sum_{\rho,\varphi}S_{\rho\zeta}x^\alpha_{\varphi\rho}\sum_{\nu,\theta}x^\alpha_{\theta\nu}(\mu\nu|\theta\varphi)-\sum_{\rho,\varphi}S_{\rho\zeta}x^\alpha_{\varphi\rho}\sum_{\theta,\nu,\sigma} x^\sigma_{\theta\nu}(\mu\varphi|\theta\nu)\right]-\sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a}\left[\sum_{\rho,\varphi}S_{\mu\rho}x^\alpha_{\rho\varphi}\sum_{\nu,\theta}x^\alpha_{\nu\theta}(\zeta\nu|\theta\varphi)-\sum_{\rho,\varphi} S_{\mu\rho}x^\alpha_{\rho\varphi}\sum_{\theta,\nu,\sigma}x^\sigma_{\nu\theta}(\zeta\varphi|\theta\nu) \right]
\end{equation}

In [45]:
# Care needs to be taken with the index order of the excitation vectors
FG1 = ( numpy.einsum('rz,mr->mz', ovlp,
                    numpy.einsum('pr,mp->mr', tda_eig_vec_ao,
                        (0.5*numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
                        -numpy.einsum('tn,mptn->mp',tda_eig_vec_ao,pqrs))
                                )
                      )
            )
FG2 = ( numpy.einsum('mr,rz->mz', ovlp,
                    numpy.einsum('rp,zp->rz', tda_eig_vec_ao,
                        (0.5*numpy.einsum('nt,zntp->zp',tda_eig_vec_ao,pqrs)
                        -numpy.einsum('nt,zptn->zp',tda_eig_vec_ao,pqrs))
                                )
                      )
            )

#FG1_try2 =( numpy.einsum('rz,mr->mz', ovlp,
#                       numpy.einsum('pr,mp->mr', tda_eig_vec_ao,
#                                    numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
#                                   )
#                      )
#            )
#FG2_try2 =( numpy.einsum('rz,mr->mz', ovlp,
#                       numpy.einsum('pr,mp->mr', tda_eig_vec_ao,
#                                    numpy.einsum('tn,mptn->mp',tda_eig_vec_ao,pqrs)
#                                   )
#                      )
#            )
#FG3_try2 =( numpy.einsum('mr,rz->mz', ovlp,
#                       numpy.einsum('rp,zp->rz', tda_eig_vec_ao,
#                                    numpy.einsum('nt,zntp->zp',tda_eig_vec_ao,pqrs)
#                                   )
#                      )
#            )
#FG4_try2 =( numpy.einsum('mr,rz->mz', ovlp,
#                       numpy.einsum('rp,zp->rz', tda_eig_vec_ao,
#                                    numpy.einsum('nt,zptn->zp',tda_eig_vec_ao,pqrs)
#                                   )
#                      )
#            )
#
#
#FG1_vlx =( numpy.einsum('rz,zm->rm', ovlp,
#                       numpy.einsum('zp,mp->zm', tda_eig_vec_ao,
#                                    numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
#                                   )
#                      )
#            )
#FG1_vlx_ovlp =( numpy.einsum('rp,mp->rm',
#                numpy.einsum('rz,zp->rp', ovlp, tda_eig_vec_ao),
#                numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
#                                   )
#            )
#
#FG2_vlx =  numpy.einsum('rz,zm->rm', ovlp,
#                       numpy.einsum('zn,nm->zm', tda_eig_vec_ao,
#                                    numpy.einsum('tp,mntp->nm',tda_eig_vec_ao,pqrs)
#                                   )
#                      )
#FG3_vlx =  numpy.einsum('mz,rz->rm', ovlp,
#                       numpy.einsum('zp,rp->rz', tda_eig_vec_ao,
#                                    numpy.einsum('tn,rntp->rp',tda_eig_vec_ao,pqrs)
#                                   )
#                      )
#FG4_vlx =  numpy.einsum('mz,rz->rm', ovlp,
#                       numpy.einsum('zn,rn->rz', tda_eig_vec_ao,
#                                    numpy.einsum('tp,rntp->rn',tda_eig_vec_ao,pqrs)
#                                   )
#                      )
print("eigenvector in MO:")
print(tda_eig_vec_as_mat)
print()
print("eigenvector in AO:")
print(tda_eig_vec_ao)
print()
# print("first term:")
# print(FG1_try2)
# print()
# print("first term:")
# print(FG1_vlx-FG2_vlx)
# print()
# F_2pdm_vlx_A = 0.5*FG1_try2 - FG2_try2 
# F_2pdm_vlx_B =-0.5*FG3_try2 + FG4_try2

# F_2pdm_vlx = F_2pdm_vlx_A + F_2pdm_vlx_B
#print(F_2pdm_vlx)
# another_vlx_2pdm_rhs = (numpy.einsum('mi,ma->ia', mo_occ, numpy.einsum('za,mz->ma', mo_vir, F_2pdm_vlx_A))
#                 +numpy.einsum('mi,ma->ia', mo_occ, numpy.einsum('za,mz->ma', mo_vir, F_2pdm_vlx_B))
#                )
# vlx_2pdm_rhs = (numpy.einsum('mi,ma->ia', mo_occ, numpy.einsum('za,mz->ma', mo_vir, F_2pdm_vlx)))
final_vlx_2pdm_rhs = (numpy.einsum('mi,ma->ia', mo_occ, numpy.einsum('za,mz->ma', mo_vir, FG1-FG2)))
# print(vlx_2pdm_rhs)
# print()
# print(another_vlx_2pdm_rhs)         
# print()
print(final_vlx_2pdm_rhs)

eigenvector in MO:
[[-0.        -0.       ]
 [-0.         0.       ]
 [-0.         0.       ]
 [ 0.         0.       ]
 [ 0.9999995 -0.0009539]]

eigenvector in AO:
[[ 0.        -0.         0.         0.        -0.        -0.
  -0.       ]
 [-0.         0.        -0.        -0.         0.         0.
   0.       ]
 [ 0.        -0.         0.         0.         0.        -0.
  -0.       ]
 [-0.         0.        -0.        -0.        -0.         0.
   0.       ]
 [-0.1356065  0.9082247 -0.805356  -0.8090212 -0.         0.4104525
   0.5846113]
 [-0.         0.        -0.        -0.        -0.         0.
   0.       ]
 [ 0.        -0.         0.         0.        -0.        -0.
  -0.       ]]

[[-0.0193612  0.0000185]
 [ 0.0193312 -0.0000184]
 [ 0.0001742 -0.0000002]
 [ 0.0580351 -0.0000554]
 [-0.        -0.       ]]


In [46]:
# Compare to adcc
adcc_2pdm_rhs = (numpy.einsum('jakb,ijkb->ia', adcc_DM_ovov, ooov, optimize=True)
                + numpy.einsum('ibjc,jcab->ia', adcc_DM_ovov, ovvv, optimize=True)
                )
print(adcc_2pdm_rhs)
#print(ooov)

[[-0.0193612  0.0000185  0.         0.       ]
 [ 0.0193312 -0.0000184  0.         0.       ]
 [ 0.0001742 -0.0000002  0.         0.       ]
 [ 0.0580351 -0.0000554  0.         0.       ]
 [ 0.        -0.         0.         0.       ]
 [ 0.         0.        -0.0193612  0.0000185]
 [ 0.         0.         0.0193312 -0.0000184]
 [ 0.         0.         0.0001742 -0.0000002]
 [ 0.         0.         0.0580351 -0.0000554]
 [ 0.         0.         0.        -0.       ]]


### INITIAL GUESS

In [47]:
# Neglect the non-diagonal part of the LHS,
# i.e. divide the RHS by the orbital-energy difference
vlx_rhs = final_vlx_2pdm_rhs + vlx_1pdm_rhs
lambda_guess = numpy.zeros((nocc,nvir))
mo_energies = scfdrv.scf_tensors['E']
#print(mo_energies[nocc:])
for i in range(nocc):
    ei=mo_energies[i]
    for a in range(nvir):
        ea=mo_energies[nocc+a]
        lambda_guess[i,a] = vlx_rhs[i,a]/(ei-ea)
print(lambda_guess)

[[ 0.0000493  0.0000003]
 [ 0.0339298  0.0000256]
 [ 0.0002304 -0.0766874]
 [ 0.0313636 -0.0000823]
 [-0.        -0.       ]]


In [48]:
# Compare to the analogous calculation with adcc
foo = adc1.reference_state.fock("o1o1").to_ndarray() 
fvv = adc1.reference_state.fock("v1v1").to_ndarray()

adcc_rhs = adcc_1pdm_rhs + adcc_2pdm_rhs
adcc_guess = adcc_rhs.copy() # rhs
for i in range(foo.shape[0]):
    for a in range(fvv.shape[0]):
        adcc_guess[i,a] = adcc_guess[i,a]/(foo[i,i]-fvv[a,a])

print(adcc_guess)

[[ 0.0000493  0.0000003 -0.        -0.       ]
 [ 0.0339298  0.0000256 -0.        -0.       ]
 [ 0.0002304 -0.0766874 -0.        -0.       ]
 [ 0.0313636 -0.0000823 -0.        -0.       ]
 [-0.        -0.        -0.        -0.       ]
 [-0.        -0.         0.0000493  0.0000003]
 [-0.        -0.         0.0339298  0.0000256]
 [-0.        -0.         0.0002304 -0.0766874]
 [-0.        -0.         0.0313636 -0.0000823]
 [-0.        -0.        -0.        -0.       ]]


In [49]:
#A L = RHS
# Transform the initial guess for lambda to the AO basis

lambda_guess_ao = numpy.matmul(mo_occ, numpy.matmul(lambda_guess, mo_vir.T))
print(lambda_guess_ao)

#print(numpy.matmul(mo_occ, numpy.matmul(adcc_guess[:nocc,:nvir], mo_vir.T))[:nocc+nvir,:nocc+nvir])

# Transform back to MO basis to check
lambda_guess_back_to_mo = numpy.matmul(mo_occ.T, numpy.matmul(ovlp, numpy.matmul(lambda_guess_ao, numpy.matmul(ovlp, mo_vir))))
print(lambda_guess_back_to_mo)

[[ 0.0006387 -0.0042777  0.0038045  0.0037992  0.        -0.0019443
  -0.0027457]
 [-0.0016019  0.010729  -0.0095546 -0.0095164 -0.         0.0048886
   0.0068779]
 [-0.0018755  0.0125678 -0.0396589  0.0173315 -0.         0.0335656
  -0.0116339]
 [-0.0018882  0.0126399  0.0173158 -0.0397953  0.        -0.0221828
   0.0278663]
 [-0.         0.         0.        -0.         0.        -0.
   0.       ]
 [-0.0022784  0.015267  -0.0451843  0.0180605 -0.         0.0378484
  -0.0120629]
 [-0.0032374  0.0216772  0.0031725 -0.0417133  0.        -0.0121041
   0.0294436]]
[[ 0.0000493  0.0000003]
 [ 0.0339298  0.0000256]
 [ 0.0002304 -0.0766874]
 [ 0.0313636 -0.0000823]
 [-0.        -0.       ]]


### LHS: Contraction of the initial guess for $\boldsymbol{\lambda}$ with two-electron integrals

wrong equation, spin not integrated out, which needs to be done at the beginning!
\begin{equation}
    \sum_{\mu \nu} C_{\mu i} C_{\nu a}
    \sum_{\lambda \sigma} \big[
    \tilde{\lambda}_{\lambda \sigma} ( \mu \nu | \sigma \lambda )
    - \tilde{\lambda}_{\lambda \sigma} ( \mu \lambda | \sigma \nu )
        \big]
    %+ (\varepsilon_a - \varepsilon_i ) \gamma_{ia}
\end{equation}

using $p=j, q=b$ as well as $p=b, q=j$ with the condition $\lambda_{pq} = \lambda_{qp}$, the probably correct one is obtained:
\begin{equation}
    -\sum_{pq} \lambda_{pq} \big( [pq|ai] - [pi|aq] \big)
    = -\sum_{\varphi \theta} C_{\varphi i} C_{\theta a} \sum_{\mu \nu}
        \lambda_{\mu \nu} \big[ 4 (\mu \nu | \varphi \theta)
        - (\mu \theta | \varphi \nu) - (\nu \theta | \varphi \mu) \big]
\end{equation}

In [50]:
vlx_lhs1 = numpy.einsum('mn,mnpt->pt', lambda_guess_ao, pqrs)
vlx_lhs2 = numpy.einsum('mn,mtpn->pt', lambda_guess_ao, pqrs)
vlx_lhs3 = numpy.einsum('mn,ntpm->pt', lambda_guess_ao, pqrs)

vlx_lhs_ao = -2*vlx_lhs1 + 0.5*vlx_lhs2 + 0.5*vlx_lhs3

# Transform to MO
vlx_lhs_mo = numpy.matmul(mo_occ.T, numpy.matmul(vlx_lhs_ao, mo_vir))
#vlx_lhs_mo2 = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, vlx_lhs_ao))
print(vlx_lhs_mo)
#print(vlx_lhs_mo2)

[[ 0.0014763  0.0000022]
 [-0.0034964 -0.0000172]
 [ 0.0000151  0.000058 ]
 [-0.0044806 -0.0000239]
 [ 0.        -0.       ]]


In [51]:
#vlx_lhs3 = numpy.einsum('mn,mntp->pt', lambda_guess_ao, pqrs)
#vlx_lhs4 = numpy.einsum('mn,mptn->pt', lambda_guess_ao, pqrs)
#vlx_lhs_ao_try2 = -vlx_lhs3 + 0.5*vlx_lhs4
#vlx_1pdm_lhs = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, vlx_lhs_ao_try2))
#print(vlx_1pdm_lhs)

### Equation derived using $p=j, q=b$ and $p=b, q=j$:

\begin{equation}
-\sum_{pq}\lambda_{pq}\langle pa||qi\rangle = -\sum_{\theta\varphi}C^\alpha_{\theta a}C^\alpha_{\varphi i} \sum_{\mu\nu}(\mu\nu|\theta\varphi)\sum_{j,b,\sigma}C^\sigma_{\mu j} \lambda_{jb} C^\sigma_{\nu b}+\sum_{\theta\varphi}C^\alpha_{\theta a}C^\alpha_{\varphi i} \sum_{\mu\nu}(\mu\varphi|\theta\nu)\sum_{j,b}C^\alpha_{\mu j} \lambda_{jb} C^\alpha_{\nu b}\\
-\sum_{\theta\varphi}C^\alpha_{\theta a}C^\alpha_{\varphi i} \sum_{\mu\nu}(\nu\mu|\theta\varphi)\sum_{j,b,\sigma}C^\sigma_{\mu j} \lambda_{jb} C^\sigma_{\nu b}+\sum_{\theta\varphi}C^\alpha_{\theta a}C^\alpha_{\varphi i} \sum_{\mu\nu}(\nu\varphi|\theta\mu)\sum_{j,b}C^\alpha_{\mu j} \lambda_{jb} C^\alpha_{\nu b}\\
\\
=-\sum_{\theta\varphi}C^\alpha_{\theta a}C^\alpha_{\varphi i} \sum_{\mu\nu}(\mu\nu|\theta\varphi)\lambda^\sigma_{\mu\nu}+\sum_{\theta\varphi}C^\alpha_{\theta a}C^\alpha_{\varphi i} \sum_{\mu\nu}(\mu\varphi|\theta\nu)\lambda^\alpha_{\mu\nu}\\
-\sum_{\theta\varphi}C^\alpha_{\theta a}C^\alpha_{\varphi i} \sum_{\mu\nu}(\nu\mu|\theta\varphi)\lambda^\sigma_{\mu\nu}+\sum_{\theta\varphi}C^\alpha_{\theta a}C^\alpha_{\varphi i} \sum_{\mu\nu}(\nu\varphi|\theta\mu)\lambda^\alpha_{\mu\nu}\\
\\
=-2\sum_{\theta\varphi}C^\alpha_{\theta a}C^\alpha_{\varphi i} \sum_{\mu\nu,\sigma}(\mu\nu|\theta\varphi)\lambda^\sigma_{\mu\nu}+\sum_{\theta\varphi}C^\alpha_{\theta a}C^\alpha_{\varphi i} \sum_{\mu\nu}(\mu\varphi|\theta\nu)\lambda^\alpha_{\mu\nu}+\sum_{\theta\varphi}C^\alpha_{\theta a}C^\alpha_{\varphi i} \sum_{\mu\nu}(\nu\varphi|\theta\mu)\lambda^\alpha_{\mu\nu}
\end{equation}

In [52]:
vlx_lhs5 = numpy.einsum('mn,mnpt->pt', lambda_guess_ao, pqrs)
vlx_lhs6 = numpy.einsum('mn,mtpn->pt', lambda_guess_ao, pqrs)
vlx_lhs7 = numpy.einsum('mn,ntpm->pt', lambda_guess_ao, pqrs)
vlx_lhs_ao_try3 = -2*vlx_lhs5 + 0.5*vlx_lhs6 + 0.5*vlx_lhs7

vlx_1pdm_lhs_new = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, vlx_lhs_ao_try3))
print(vlx_1pdm_lhs_new)

[[ 0.0014763  0.0000022]
 [-0.0034964 -0.0000172]
 [ 0.0000151  0.000058 ]
 [-0.0044806 -0.0000239]
 [ 0.        -0.       ]]


In [53]:
print("vlx_lhs_mo:")
print(vlx_lhs_mo)
print()
print("lambda_guess:")
print(lambda_guess)

vlx_lhs_mo:
[[ 0.0014763  0.0000022]
 [-0.0034964 -0.0000172]
 [ 0.0000151  0.000058 ]
 [-0.0044806 -0.0000239]
 [ 0.        -0.       ]]

lambda_guess:
[[ 0.0000493  0.0000003]
 [ 0.0339298  0.0000256]
 [ 0.0002304 -0.0766874]
 [ 0.0313636 -0.0000823]
 [-0.        -0.       ]]


In [54]:
# Add product of guess and eigenvalue differences to LHS
vlx_full_lhs = numpy.zeros((nocc,nvir))
for i in range(nocc):
    ei = mo_energies[i]
    for a in range(nvir):
        ea = mo_energies[nocc+a]
        vlx_full_lhs[i,a] = vlx_lhs_mo[i,a] + lambda_guess[i,a]*(ei-ea)

vlx_full_lhs_vec = vlx_full_lhs.reshape(nocc*nvir)
vlx_rhs_vec = vlx_rhs.reshape(nocc*nvir)

print(vlx_full_lhs_vec)
print()
print(vlx_rhs_vec)

[ 0.000448  -0.0000042 -0.0668962 -0.0000692 -0.0002688  0.1070512
 -0.03724    0.0000755  0.         0.       ]

[-0.0010282 -0.0000064 -0.0633997 -0.000052  -0.0002839  0.1069932
 -0.0327595  0.0000994  0.         0.       ]


In [55]:
from scipy.sparse import linalg

In [56]:
# Try conjugate gradient with identity matrix, full LHS and RHS
# identity = numpy.identity(nocc*nvir)

# solution, w = linalg.cg(A=identity, b=vlx_rhs_vec, x0=vlx_full_lhs_vec, tol=1e-6, maxiter=25)

# print(solution)
# print()
# print(w)

### Trying to define a linear Operator

In [57]:
# class vlx_lambda:
#     def __init__(self, guess, eri, scf_tensors, nocc):
#         """Parameters:
#         vlx_lambda: guess at current iteration for the lambda OV block
#         eri: two electron repulsion integrals in AO basis (chemists notation)
#         scf_tensors: from vlx scdrv
#         nocc: number of occupied orbitals
#         """
#         self.vector = guess
#         self.eri = eri
#         self.scf_tensors = scf_tensors
#         #print(scf_tensors)
#         self.nocc = nocc
        
def Ax(v):
    """Function to carry out matrix multiplication
    of Lagrange multipier vector with orbital Hessian
    matrix. Transforms guess to AO basis, carries out
    matrix-vector product and transforms back to MO.
    
    v: lambda at current iteration"""
    #print("Entering Ax(v)...:")
    #print(v)
    #print()
    #vlx_1pdm_lhs = 2.0*v
    #mo_energies = v.scf_tensors['E']
    #mo_occ = v.scf_tensors['C'][:,:nocc]
    #mo_vir = v.scf_tensors['C'][:,nocc:]
    #nvir = mo_vir.shape[1]
    current_lambda = v.reshape(nocc,nvir)
    print("VLX INPUT:")
    print(current_lambda)
    print()
    
    #vlx_lambda_2_in_ao = numpy.matmul(mo_occ, numpy.matmul(new_v, mo_vir.T))
    #L1_vlx = numpy.einsum('mn,mntp->pt', vlx_lambda_2_in_ao, pqrs)
    #L2_vlx = numpy.einsum('mn,mptn->pt', vlx_lambda_2_in_ao, pqrs)
    #L3_vlx = numpy.einsum('mn,nmtp->pt', vlx_lambda_2_in_ao, pqrs)
    #L4_vlx = numpy.einsum('mn,nptm->pt', vlx_lambda_2_in_ao, pqrs)
    #L_total_vlx = -2.0*L1_vlx + 1.0*L2_vlx - 2.0*L3_vlx + 1.0*L4_vlx
    #vlx_lambda_lhs_2 = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, L_total_vlx))
    #print("Vlx output using 4 terms:")
    #print(vlx_lambda_lhs_2)
    
    # Transform to AO
    vlx_lambda_ao = numpy.matmul(mo_occ, numpy.matmul(current_lambda, mo_vir.T))
 
    # Calculate the different contractions
    vlx_lhs_interm = (-4*numpy.einsum('mn,mntp->pt', vlx_lambda_ao, pqrs)
                    + 1.0*numpy.einsum('mn,mtpn->pt', vlx_lambda_ao, pqrs)
                    + 1.0*numpy.einsum('mn,ntpm->pt', vlx_lambda_ao, pqrs)
                      )
    # Transform back to MO
    vlx_1pdm_lhs = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, vlx_lhs_interm))

# Add the diagonal part
    for i in range(nocc):
        ei = mo_energies[i]
        for a in range(nvir):
            ea = mo_energies[nocc+a]
            vlx_1pdm_lhs[i,a] += current_lambda[i,a]*(ei-ea) 
    print("VLX OUTPUT:")    
    print(vlx_1pdm_lhs)
    print()
    return vlx_1pdm_lhs.reshape(nocc*nvir)
    

In [58]:
#new_lambda_guess = vlx_lambda(lambda_guess, pqrs, scfdrv.scf_tensors, nocc)
# print("This is lambda guess")
vlx_guess_vec=lambda_guess.reshape(nocc*nvir)
# print(vlx_guess_vec)
# print()
# print("This is the rhs:")
# print(vlx_rhs_vec)
# print()
# print("This is Ax(lambda_guess)")
# print(Ax(v).shape)
# print()
A = linalg.LinearOperator((nocc*nvir,nocc*nvir), matvec=Ax)
solution, w = linalg.cg(A=A, b=vlx_rhs_vec, x0=vlx_guess_vec, tol=1e-8, maxiter=25)

print("The solution in matrix form:")
print(solution.reshape(nocc,nvir))

VLX INPUT:
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]

VLX OUTPUT:
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]

VLX INPUT:
[[ 0.0000493  0.0000003]
 [ 0.0339298  0.0000256]
 [ 0.0002304 -0.0766874]
 [ 0.0313636 -0.0000823]
 [-0.        -0.       ]]

VLX OUTPUT:
[[ 0.0019243 -0.000002 ]
 [-0.0703926 -0.0000864]
 [-0.0002537  0.1071092]
 [-0.0417206  0.0000516]
 [ 0.         0.       ]]

VLX INPUT:
[[ 0.0000493  0.0000003]
 [ 0.0339298  0.0000256]
 [ 0.0002304 -0.0766874]
 [ 0.0313636 -0.0000823]
 [-0.        -0.       ]]

VLX OUTPUT:
[[ 0.0019243 -0.000002 ]
 [-0.0703926 -0.0000864]
 [-0.0002537  0.1071092]
 [-0.0417206  0.0000516]
 [ 0.         0.       ]]

VLX INPUT:
[[-0.0029525 -0.0000044]
 [ 0.0069929  0.0000344]
 [-0.0000302 -0.000116 ]
 [ 0.0089611  0.0000478]
 [-0.         0.       ]]

VLX OUTPUT:
[[ 0.0594659  0.0000882]
 [-0.0120036 -0.0000557]
 [ 0.0000224  0.0033326]
 [-0.0071516 -0.0000399]
 [ 0.        -0.       ]]

VLX INPUT:
[[ 0.0107969  0.0000161]
 [ 0.0292796  0.

In [59]:
AOVOV = numpy.ones(ovov.shape)   
adcc_nocc = foo.shape[0]
adcc_nvirt = fvv.shape[0]
oovv = adc1.reference_state.eri("o1o1v1v1").to_ndarray()
#adcc_A = np.zeros((adcc_nocc*adcc_nvirt, adcc_nocc*adcc_nvirt))
#these are supposed to be the elements that multiply lambda_jb
AOVOV = ( numpy.einsum('iajb,ibja->iajb', AOVOV, ovov, optimize=True)
         -numpy.einsum('iajb,ijab->iajb', AOVOV, oovv, optimize=True) 
        )
#print("OOVV\n")
#print(oovv)
#print()
for i in range(adcc_nocc):
    for a in range(adcc_nvirt):
        AOVOV[i,a,i,a] += foo[i,i]-fvv[a,a] #check signs!

adcc_A = AOVOV.reshape(adcc_nocc*adcc_nvirt,adcc_nocc*adcc_nvirt)

In [60]:
adcc_1d_guess = numpy.reshape(adcc_guess, (adcc_nocc*adcc_nvirt))
adcc_1d_rhs = numpy.reshape(adcc_rhs, (adcc_nocc*adcc_nvirt))

# print("MATRIX A:\n")
# print(adcc_A)
# print()
# print("GUESS:\n")
# print(adcc_1d_guess)
# print()
# print("RHS:\n")
# print(adcc_1d_rhs)
# print()

lov, w = linalg.cg(A=adcc_A, b=adcc_1d_rhs, x0=adcc_1d_guess, tol=1e-8, maxiter=25)

In [61]:
def adcc_Ax_MO(v):
    print("Entering Ax_MO(v)...:")
    #print(v)
    #print()
    new_v = v.reshape(adcc_nocc,adcc_nvirt)
    print("ADCC INPUT")
    print(new_v)
    print()
    adcc_1pdm_lhs = ( numpy.einsum('kb,kaib->ia', new_v, ovov, optimize=True)
         -numpy.einsum('kb,kiba->ia', new_v, oovv, optimize=True) 
        )

    # Add the diagonal part
    for i in range(adcc_nocc):
        ei = foo[i,i]
        for a in range(adcc_nvirt):
            ea = fvv[a,a]
            adcc_1pdm_lhs[i,a] += new_v[i,a]*(ei-ea)
    print("ADCC OUTPUT")
    print(adcc_1pdm_lhs)
    print()
    return adcc_1pdm_lhs.reshape(adcc_nocc*adcc_nvirt)

adcc_Ax = linalg.LinearOperator((adcc_nocc*adcc_nvirt,adcc_nocc*adcc_nvirt), matvec=adcc_Ax_MO)
adcc_solution, adcc_w = linalg.cg(A=adcc_Ax, b=adcc_1d_rhs, x0=adcc_1d_guess, tol=1e-8, maxiter=25)

Entering Ax_MO(v)...:
ADCC INPUT
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

ADCC OUTPUT
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

Entering Ax_MO(v)...:
ADCC INPUT
[[ 0.0000493  0.0000003 -0.        -0.       ]
 [ 0.0339298  0.0000256 -0.        -0.       ]
 [ 0.0002304 -0.0766874 -0.        -0.       ]
 [ 0.0313636 -0.0000823 -0.        -0.       ]
 [-0.        -0.        -0.        -0.       ]
 [-0.        -0.         0.0000493  0.0000003]
 [-0.        -0.         0.0339298  0.0000256]
 [-0.        -0.         0.0002304 -0.0766874]
 [-0.        -0.         0.0313636 -0.0000823]
 [-0.        -0.        -0.        -0.       ]]

ADCC OUTPUT
[[ 0.0019243 -0.000002   0.         0.       ]
 [-0.0703926 -0.0000864  0.         0.       ]
 [-0.0002537  0.1071092  0.         0.  

ADCC OUTPUT
[[-0.         0.         0.         0.       ]
 [ 0.        -0.         0.         0.       ]
 [ 0.0000001  0.         0.         0.       ]
 [-0.         0.0000001  0.         0.       ]
 [-0.         0.         0.         0.       ]
 [ 0.         0.        -0.         0.       ]
 [ 0.         0.         0.        -0.       ]
 [ 0.         0.         0.0000001  0.       ]
 [ 0.         0.        -0.         0.0000001]
 [ 0.         0.        -0.         0.       ]]

Entering Ax_MO(v)...:
ADCC INPUT
[[-0.  0.  0.  0.]
 [ 0. -0.  0.  0.]
 [-0. -0.  0.  0.]
 [ 0. -0.  0.  0.]
 [ 0. -0.  0.  0.]
 [ 0.  0. -0.  0.]
 [ 0.  0.  0. -0.]
 [ 0.  0. -0. -0.]
 [ 0.  0.  0. -0.]
 [ 0.  0.  0. -0.]]

ADCC OUTPUT
[[ 0. -0.  0.  0.]
 [-0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [-0. -0.  0.  0.]
 [-0.  0.  0.  0.]
 [ 0.  0.  0. -0.]
 [ 0.  0. -0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0. -0. -0.]
 [ 0.  0. -0.  0.]]

Entering Ax_MO(v)...:
ADCC INPUT
[[ 0.0001931  0.0000005 -0.        -0.       ]
 [ 0.03

In [62]:
print("MO basis, ADCC solution, regular CG")
print(lov.reshape(adcc_nocc,adcc_nvirt))
print()
print("MO basis, ADCC solution, Linear Operator")
print(adcc_solution.reshape(adcc_nocc, adcc_nvirt))
print()
print("AO basis, VLX solution, linear operator")
print(solution.reshape(nocc,nvir))

MO basis, ADCC solution, regular CG
[[ 0.0001931  0.0000005 -0.        -0.       ]
 [ 0.0302105  0.0000041 -0.        -0.       ]
 [ 0.0002684 -0.0793038 -0.        -0.       ]
 [ 0.0182954 -0.0001408 -0.        -0.       ]
 [-0.        -0.        -0.        -0.       ]
 [-0.        -0.         0.0001931  0.0000005]
 [-0.        -0.         0.0302105  0.0000041]
 [-0.        -0.         0.0002684 -0.0793038]
 [-0.        -0.         0.0182954 -0.0001408]
 [-0.        -0.        -0.        -0.       ]]

MO basis, ADCC solution, Linear Operator
[[ 0.0001931  0.0000005 -0.        -0.       ]
 [ 0.0302105  0.0000041 -0.        -0.       ]
 [ 0.0002684 -0.0793038 -0.        -0.       ]
 [ 0.0182954 -0.0001408 -0.        -0.       ]
 [-0.        -0.        -0.        -0.       ]
 [-0.        -0.         0.0001931  0.0000005]
 [-0.        -0.         0.0302105  0.0000041]
 [-0.        -0.         0.0002684 -0.0793038]
 [-0.        -0.         0.0182954 -0.0001408]
 [-0.        -0.        -0. 

In [63]:
#solution, w = linalg.cg(A=A, b=vlx_rhs_vec, x0=vlx_guess_vec, tol=1e-8, maxiter=25)
#adcc_solution, adcc_w = linalg.cg(A=adcc_Ax, b=adcc_1d_rhs, x0=adcc_1d_guess, tol=1e-8, maxiter=25)
print("VLX RHS:\n")
print(vlx_rhs_vec)
print()
print("ADCC RHS:\n")
print(adcc_1d_rhs)
print()
print("VLX GUESS:\n")
print(vlx_guess_vec)
print()
print("ADCC GUESS:\n")
print(adcc_1d_guess)
print()
vlx_apply_Ax = Ax(vlx_guess_vec)
adcc_apply_Ax = adcc_Ax_MO(adcc_1d_guess)
print("VLX Ax(v)\n")
print(vlx_apply_Ax)
print()
print("ADCC Ax(v)\n")
print(adcc_apply_Ax)
print()

VLX RHS:

[-0.0010282 -0.0000064 -0.0633997 -0.000052  -0.0002839  0.1069932
 -0.0327595  0.0000994  0.         0.       ]

ADCC RHS:

[-0.0010282 -0.0000064  0.         0.        -0.0633997 -0.000052
  0.         0.        -0.0002839  0.1069932  0.         0.
 -0.0327595  0.0000994  0.         0.         0.         0.
  0.         0.         0.         0.        -0.0010282 -0.0000064
  0.         0.        -0.0633997 -0.000052   0.         0.
 -0.0002839  0.1069932  0.         0.        -0.0327595  0.0000994
  0.         0.         0.         0.       ]

VLX GUESS:

[ 0.0000493  0.0000003  0.0339298  0.0000256  0.0002304 -0.0766874
  0.0313636 -0.0000823 -0.        -0.       ]

ADCC GUESS:

[ 0.0000493  0.0000003 -0.        -0.         0.0339298  0.0000256
 -0.        -0.         0.0002304 -0.0766874 -0.        -0.
  0.0313636 -0.0000823 -0.        -0.        -0.        -0.
 -0.        -0.        -0.        -0.         0.0000493  0.0000003
 -0.        -0.         0.0339298  0.0000256 